# Simple NN with pooling

In [1]:
from sklearn.metrics import f1_score
import pandas as pd
import datautils
import simpleNN

In [2]:
PATH = './Dataset/power-gb-train.tsv'
RES_DIR = './Results/'
EMBED_DIR = './Embeddings/'

## Tf-idf embeddings

In [ ]:
X_train, y_train, X_val, y_val, _, _ = datautils.split_kfold_dataset(PATH)

for idx, (tr_fold, val_fold) in enumerate(zip(X_train, X_val)):

    X_train[idx], vectorizer = datautils.tf_idf_preprocessing(tr_fold)
    X_val[idx] = vectorizer.transform(val_fold)

In [ ]:
# train simple NN here

## Word2vec embeddings

In [9]:
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess

In [10]:
X_train, y_train, X_val, y_val, _, _ = datautils.split_kfold_dataset(PATH)

for idx, (tr_fold, val_fold) in enumerate(zip(X_train, X_val)):
    word_vectors = KeyedVectors.load(EMBED_DIR+f'w2v-fold{idx}.kv', mmap='r')

    tr_fold = list(map(simple_preprocess, tr_fold))
    val_fold = list(map(simple_preprocess, val_fold))

    X_train[idx] = datautils.documents_vector_wv(tr_fold,word_vectors)
    X_val[idx] = datautils.documents_vector_wv(val_fold,word_vectors)

In [8]:
# train simple NN here

True

## Fasttext embeddings

In [1]:
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess

In [ ]:
X_train, y_train, X_val, y_val, _, _ = datautils.split_kfold_dataset(PATH)

for idx, (tr_fold, val_fold) in enumerate(zip(X_train, X_val)):
    word_vectors = KeyedVectors.load(EMBED_DIR+f'w2v-fold{idx}.kv', mmap='r')

    tr_fold = list(map(simple_preprocess, tr_fold))
    val_fold = list(map(simple_preprocess, val_fold))

    X_train[idx] = datautils.documents_vector_wv(tr_fold,word_vectors)
    X_val[idx] = datautils.documents_vector_wv(val_fold,word_vectors)

In [ ]:
#train simple NN here